In [1]:
%load_ext autoreload
%autoreload 1
import sys
sys.path.append(r'D:\\CodeRepository\\Python\\IDMSImplementation')
%aimport idms_impl.trajectory_generator
%aimport idms_impl.stop_area_mining
%aimport idms_impl.semantic_tag_conversion

In [2]:
orig_filename = r'C:\Users\Scott\Documents\ECNU\data\20201116\9_user.csv'

usecols = ['SUBSTR(MSISDN,1,7)', 'STAT_DATE', 'STIME', 'END_TIME', 'ZH_LABEL', 'LATITUDE', 'NUMBERITUDE', 'DURATION', 'DATA_TOTAL']
name_mapper = {
    "SUBSTR(MSISDN,1,7)": "USER_ID",
    "NUMBERITUDE": "LONGITUDE",
    "DATA_TOTAL": "TOTAL_DATA"
}
%run -m idms_impl.trajectory_generator

import time

start_time = time.time()

tr_gen = TrajectoryGenerator()

tr_gen.stop_area_mining(filename=orig_filename,
                        usecols=usecols,
                        name_mapper=name_mapper,
                        nan_dur_theta=4 * 3600,
                        dist_theta=5 / 111,
                        point_dur_theta=20,
                        eps=0.0036,
                        min_dur=900)
# tr_gen.sam.load_data(filename=orig_filename)

# tr_gen.sam.format_raw_data(usecols=usecols, name_mapper={
#     "SUBSTR(MSISDN,1,7)": "USER_ID",
#     "NUMBERITUDE": "LONGITUDE",
#     "DATA_TOTAL": "TOTAL_DATA"
# })

# tr_gen.sam.check_invalid_tr(nan_dur_theta=4 * 3600, dist_theta=5 / 111)

# tr_gen.sam.gen_valid_area(point_dur_theta=20, eps=0.0036, min_dur=900)

# tr_gen.sam.df.to_csv('../data/no_invalid_area.csv', encoding='gbk', index=False)
# tr_gen.sam.merge_adjacent_points()

# tr_gen.sam.df.to_csv('../data/sam.csv', encoding='gbk', index=False)

end_time = time.time()
print(f'Finish it in {end_time - start_time}s')

Finish it in 8.49606466293335s


In [ ]:
df = tr_gen.df.copy()

In [4]:
tr_gen.semantic_tr_convertion()
tr_gen.stc.main_area_mining(0.9)

In [5]:
tr_gen.stc.cluster_attr.head(50)

sn   dur_sum             d   type
USER_ID CLUSTER_ID                                     
1377032 0           21.0  858715.0  5.704966e+04   home
        2           15.0  619153.0  2.381377e+05   work
        22           3.0   96684.0  2.282841e+05  other
        6            1.0   42723.0  4.337453e+05  other
        10           1.0   34091.0  1.371225e+05  other
        18           1.0   13850.0  7.231661e+04  other
        1            1.0    9675.0  6.273438e+00  other
        16           1.0    8490.0  8.855839e+03  other
        14           1.0    8063.0  1.029362e+05  other
        4            1.0    7159.0  5.760264e+02  other
        3            1.0    3082.0  0.000000e+00  other
        12           1.0    2043.0  5.350547e+02  other
        23           1.0    2020.0  4.192142e+03  other
        17           1.0    1953.0  1.995433e+05  other
        20           1.0    1947.0  3.136719e+00  other
        19           1.0    1854.0  1.251055e+02  other
        15           1.0    1618.0  5.035967e+02  other
        7            1.0    1557.0  0.000000e+00  other
        21           1.0     941.0  4.130105e+03  other
1381404 0           20.0  757234.0  4.015407e+04   home
        9           14.0  591426.0  6.062908e+05  other
        15           1.0   21851.0  7.684641e+04  other
        10           2.0   13607.0  3.503865e+04  other
        20           1.0   10555.0  5.609350e+03  other
        18           1.0    7730.0  5.408926e+03  other
        12           1.0    7695.0  4.831516e+03  other
        16           1.0    5735.0  9.697400e+04  other
        3            1.0    3361.0  1.781191e+02  other
        19           1.0    3240.0  1.214035e+04  other
        8            1.0    2324.0  1.747432e+03  other
        1            1.0    2259.0  3.341584e+03  other
        7            1.0    2254.0  1.070352e+02  other
        17           1.0    1920.0  5.879643e+03  other
        13           1.0    1131.0  2.663611e+04  other
        11           1.0     935.0  1.129902e+04  other
1385177 0           16.0  790589.0  1.176105e+05   home
        10           9.0  256136.0  1.028829e+06   work
        13           4.0   44321.0  1.514873e+04   work
        12           2.0   43582.0  2.046330e+06  other
        8            5.0   14642.0  4.301214e+04  other
        22           1.0   14491.0  1.628245e+05  other
        24           1.0   10022.0  6.060816e+03  other
        9            2.0    7853.0  6.340904e+04  other
        14           2.0    7089.0  6.285156e+00  other
        16           1.0    6764.0  1.921491e+05  other
        26           1.0    6516.0  9.480469e+00  other
        6            2.0    6214.0  2.089173e+03  other
        25           1.0    4225.0  1.828277e+04  other
        19           1.0    2851.0  0.000000e+00  other
        17           1.0    2384.0  0.000000e+00  other

In [201]:
user_grp = df.groupby(['USER_ID'], sort=False)

def gen_cluster_attr(user):
    cluster_grp = user.groupby(['CLUSTER_ID'], sort=False)
    attr = cluster_grp.apply(lambda x: pd.Series([x['STAT_DATE'].nunique(), x['DURATION'].sum(), x['TOTAL_DATA'].sum() / x['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd']))
    attr.sort_values('dur_sum', ascending=False, inplace=True)
    return attr

cluster_attr = user_grp.apply(gen_cluster_attr)
cluster_attr['type'] = np.array(['other'] * cluster_attr.shape[0])

def home_prob(cid, n):
    sn = cluster_attr.loc[cid, 'sn']
    sn_prob = sn / n 
    d = cluster_attr.loc[cid, 'd']
    sum_d = cluster_attr.loc[cid[0], 'd'].sum()
    d_prob = 0. if sum_d == 0. else d / sum_d
    return sn_prob + (1 - d_prob)

def gen_main_area(user):
    
    n = user['STAT_DATE'].nunique()
    uid = user.iloc[0, 0]
    dur_sum_in_user = cluster_attr.loc[uid, 'dur_sum'].sum()
    
    cur_cluster_attr = cluster_attr.loc[uid]

    sr_list = []
    tmp_dur_sum = 0
    for cluster_id, row in cur_cluster_attr.T.iteritems():
        tmp_dur_sum += row['dur_sum']
        if tmp_dur_sum / dur_sum_in_user <= theta:
            sr_list.append((uid, cluster_id))
    n_sr = len(sr_list)
    # print(sr_list)

    if n_sr == 1:
        cluster_attr.loc[sr_list[0], 'type'] = 'home'
    elif n_sr == 2:
        if home_prob(sr_list[0], n) > home_prob(sr_list[1], n):
            cluster_attr.loc[sr_list[0], 'type'] = 'home'
            cluster_attr.loc[sr_list[1], 'type'] = 'work'
        else:
            cluster_attr.loc[sr_list[0], 'type'] = 'work'
            cluster_attr.loc[sr_list[1], 'type'] = 'home'
    elif n_sr == 3:
        d0 = cluster_attr.loc[sr_list[0], 'd']
        d1 = cluster_attr.loc[sr_list[1], 'd']
        d2 = cluster_attr.loc[sr_list[2], 'd']
        if d0 > (d1 + d2) / 2:
            cluster_attr.loc[sr_list[0], 'type'] = 'work'
            cluster_attr.loc[sr_list[1], 'type'] = 'home'
            cluster_attr.loc[sr_list[2], 'type'] = 'home'
        else:
            cluster_attr.loc[sr_list[0], 'type'] = 'home'
            cluster_attr.loc[sr_list[1], 'type'] = 'work'
            cluster_attr.loc[sr_list[2], 'type'] = 'work'
        
theta = 0.9

# for uid, user in user_grp:
#     gen_main_area(user)
user_grp.apply(gen_main_area)


other


sn   dur_sum             d   type
USER_ID CLUSTER_ID                                     
1377032 0           21.0  858715.0  5.704966e+04   home
        2           15.0  619153.0  2.381377e+05   work
        22           3.0   96684.0  2.282841e+05  other
        6            1.0   42723.0  4.337453e+05  other
        10           1.0   34091.0  1.371225e+05  other
        18           1.0   13850.0  7.231661e+04  other
        1            1.0    9675.0  6.273438e+00  other
        16           1.0    8490.0  8.855839e+03  other
        14           1.0    8063.0  1.029362e+05  other
        4            1.0    7159.0  5.760264e+02  other
        3            1.0    3082.0  0.000000e+00  other
        12           1.0    2043.0  5.350547e+02  other
        23           1.0    2020.0  4.192142e+03  other
        17           1.0    1953.0  1.995433e+05  other
        20           1.0    1947.0  3.136719e+00  other
        19           1.0    1854.0  1.251055e+02  other
        15           1.0    1618.0  5.035967e+02  other
        7            1.0    1557.0  0.000000e+00  other
        21           1.0     941.0  4.130105e+03  other
1381404 0           20.0  757234.0  4.015407e+04   home
        9           14.0  591426.0  6.062908e+05  other
        15           1.0   21851.0  7.684641e+04  other
        10           2.0   13607.0  3.503865e+04  other
        20           1.0   10555.0  5.609350e+03  other
        18           1.0    7730.0  5.408926e+03  other
        12           1.0    7695.0  4.831516e+03  other
        16           1.0    5735.0  9.697400e+04  other
        3            1.0    3361.0  1.781191e+02  other
        19           1.0    3240.0  1.214035e+04  other
        8            1.0    2324.0  1.747432e+03  other
        1            1.0    2259.0  3.341584e+03  other
        7            1.0    2254.0  1.070352e+02  other
        17           1.0    1920.0  5.879643e+03  other
        13           1.0    1131.0  2.663611e+04  other
        11           1.0     935.0  1.129902e+04  other
1385177 0           16.0  790589.0  1.176105e+05   home
        10           9.0  256136.0  1.028829e+06   work
        13           4.0   44321.0  1.514873e+04   work
        12           2.0   43582.0  2.046330e+06  other
        8            5.0   14642.0  4.301214e+04  other
        22           1.0   14491.0  1.628245e+05  other
        24           1.0   10022.0  6.060816e+03  other
        9            2.0    7853.0  6.340904e+04  other
        14           2.0    7089.0  6.285156e+00  other
        16           1.0    6764.0  1.921491e+05  other
        26           1.0    6516.0  9.480469e+00  other
        6            2.0    6214.0  2.089173e+03  other
        25           1.0    4225.0  1.828277e+04  other
        19           1.0    2851.0  0.000000e+00  other
        17           1.0    2384.0  0.000000e+00  other

In [151]:
cluster_attr.head(20)

sn   dur_sum              d
USER_ID CLUSTER_ID                               
1377032 0           21.0  858715.0   57049.663830
        2           15.0  619153.0  238137.694661
        22           3.0   96684.0  228284.142578
        6            1.0   42723.0  433745.323242
        10           1.0   34091.0  137122.463867
        18           1.0   13850.0   72316.613281
        1            1.0    9675.0       6.273438
        16           1.0    8490.0    8855.838867
        14           1.0    8063.0  102936.178711
        4            1.0    7159.0     576.026367
        3            1.0    3082.0       0.000000
        12           1.0    2043.0     535.054688
        23           1.0    2020.0    4192.141602
        17           1.0    1953.0  199543.294922
        20           1.0    1947.0       3.136719
        19           1.0    1854.0     125.105469
        15           1.0    1618.0     503.596680
        7            1.0    1557.0       0.000000
        21           1.0     941.0    4130.105469
1381404 0           20.0  757234.0   40154.070947

In [ ]:

# cluster_grp = df.groupby(['USER_ID', 'CLUSTER_ID'], sort=False)
# def func(cluster):
#     series = pd.Series([cluster['STAT_DATE'].nunique(), cluster['DURATION'].sum(), cluster['TOTAL_DATA'].sum() / cluster['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd'])
#     return series
# cluster_attr = cluster_grp.apply(func)


user_grp = df.groupby(['USER_ID'], sort=False)

def func(user):

    c_grp = user.groupby(['CLUSTER_ID'], sort=False)
    
    attr = c_grp.apply(lambda x: pd.Series([x['STAT_DATE'].nunique(), x['DURATION'].sum(), x['TOTAL_DATA'].sum() / x['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd']))

    attr.sort_values('dur_sum', ascending=False, inplace=True)
    return attr

cluster_attr = user_grp.apply(func)
cluster_attr.head(50)
# cluster_attr.index

# print(cluster_attr.index.names)
# cluster_attr.loc['USER_ID']
# cluster_attr[['sn', 'dur_sum']] = cluster_attr[['sn', 'dur_sum']].astype(int)

# cluster_attr.sort_values('dur_sum', ascending=False, inplace=True)
# cluster_attr.head(50)
# cluster_attr.loc[(1377032, 0), 'sn']
# for cid, cluster in cluster_grp:
#     # print(cluster['STAT_DATE'])
#     print(cluster['STAT_DATE'].nunique())
#     # print(cluster['DURATION'])
#     print(cluster['DURATION'].sum())
#     print(cluster['TOTAL_DATA'].sum())
#     print(cluster['TOTAL_DATA'].sum() / cluster['STAT_DATE'].nunique())
#     break

In [ ]:
from typing import Tuple
def home_prob(cid, n):
    sn = cluster_attr.loc[cid, 'sn']
    sn_prob = sn / n 
    d = cluster_attr.loc[cid, 'd']
    sum_d = cluster_attr.loc[cid[0], 'd'].sum()
    d_prob = 0. if sum_d == 0. else d / sum_d
    print(d_prob)
    print(sn_prob)
    return sn_prob + (1 - d_prob)
    # sn_prob = cluster_attr.loc[cid, 'sn']
home_prob((1377032, 0), 21)
# print(cluster_attr.loc[1377032, 'd'].sum())


In [ ]:
user_grp = df.groupby(['USER_ID'], sort=False)

for uid, user in user_grp:
    # c_grp = user.groupby(['CLUSTER_ID'], sort=False)
    cluster = cluster_attr.loc[uid]
    sss = cluster['dur_sum'].sum()
    print(sss)
    dur_sums = cluster['dur_sum'].values
    # print(dur_sums)
    # print(cluster.iloc[[0,1, 2]].index)
    dd = 0
    for i, dur_sum in enumerate( dur_sums):
        print(i, dur_sum)
    break